In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.linear_model import ElasticNet

In [2]:
df = pd.read_csv('Datamatrix_gpd.csv')
df_array = np.asarray(df,dtype='float32')
friction = df_array[0:2401,0]
psize = df_array[0:2401,1]
gsize = df_array[0:2401,2]
spreadspe = df_array[0:2401,3]
MFR = df_array[0:2401,4]
pac = df_array[0:2401,5]
frictionsin = np.sin(friction)
psizesin = np.sin(psize)
gsizesin = np.sin(gsize)
spreadspesin = np.sin(spreadspe)
frictionsquare = np.square(friction)
psizesquare = np.square(psize)
gsizesquare = np.square(gsize)
spreadspesquare = np.square(spreadspe)
frictionlog = np.log(friction)
psizelog = np.log(psize)
gsizelog = np.log(gsize)
spreadspelog = np.log(spreadspe)

X = np.atleast_2d([friction,psize,gsize,spreadspe,frictionsin,psizesin,gsizesin,spreadspesin,frictionsquare,psizesquare,gsizesquare,spreadspesquare,frictionlog,psizelog,gsizelog,spreadspelog]).T
y = np.atleast_2d([pac]).T


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [4]:
def test(models, X_train, X_test, y_train, y_test, iterations = 100):
    results = {}
    for i in models:
        r2_train = []
        r2_test = []
        for j in range(iterations):

            r2_test.append(metrics.r2_score(y_test,
                                            models[i].fit(X_train, 
                                                         y_train).predict(X_test)))
            r2_train.append(metrics.r2_score(y_train, 
                                             models[i].fit(X_train, 
                                                          y_train).predict(X_train)))
        results[i] = [np.mean(r2_train), np.mean(r2_test)]
    return pd.DataFrame(results)

In [5]:
models = {'OLS': linear_model.LinearRegression(),
         'Lasso': linear_model.Lasso(),
         'Ridge': linear_model.Ridge(),
         'Net': linear_model.ElasticNet(),}

In [6]:
lasso_params = {'alpha':[0.02, 0.024, 0.025, 0.026, 0.03,.05,.1,.25,.5,1]}
ridge_params = {'alpha':[1,10,50,100,200, 230, 250,265, 270, 275, 290, 300, 500]}
net_params = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]}


models2 = {'OLS': linear_model.LinearRegression(),
           'Lasso': GridSearchCV(linear_model.Lasso(), 
                               param_grid=lasso_params).fit(X, y).best_estimator_,
           'Ridge': GridSearchCV(linear_model.Ridge(), 
                               param_grid=ridge_params).fit(X, y).best_estimator_,
           'Net': GridSearchCV(linear_model.ElasticNet(), 
                               param_grid=net_params).fit(X, y).best_estimator_,}

C:\Users\donm2\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.026627693325281143, tolerance: 0.0002987737534567714
  model = cd_fast.enet_coordinate_descent(
C:\Users\donm2\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010814986191689968, tolerance: 0.00031008091173134744
  model = cd_fast.enet_coordinate_descent(
C:\Users\donm2\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.006077608559280634, tolerance: 0.0003256270429119468
  model = cd_fast.enet_coordinate_descent(


In [7]:
test(models, X_train, X_test, y_train, y_test)

,OLS,Lasso,Ridge,Net
0,0.660509,0.000000,0.659569,0.000000
1,0.633793,-0.001071,0.634894,-0.001071


In [8]:
test(models2, X_train, X_test, y_train, y_test)

,OLS,Lasso,Ridge,Net
0,0.660509,0.595867,0.659569,0.659510
1,0.633793,0.561406,0.634894,0.634779
